In [1]:
import numpy as np
import nibabel as nib
import pandas as pd
from tqdm import tqdm

In [2]:
!ls

AUT_subjects.csv			map_logq.nii.gz
BNA-maxprob-thr0-1mm.nii.gz		newClusterMain-HO_Atlas.ipynb
C1MinusC2.nii.gz			Pvals.nii.gz
Literature_Survey_ResultsSheet3.csv	Qvals.nii.gz
Literature_Survey_ResultsSheet3old.csv	results_consistency_check.ipynb
map_logp.nii.gz				TD_subects.csv
map_logq_2mm.nii.gz			Tvals.nii.gz


In [3]:
brain = nib.load('map_logq.nii.gz').get_data()

In [4]:
brain.shape

(61, 73, 61, 274)

In [7]:
brain[43,30,30,31]

0.01668051

In [8]:
# Read Brainnetome atlas
atlas_path = 'BNA-maxprob-thr0-1mm.nii.gz'
atlas = nib.load(atlas_path).get_data()

In [9]:
np.min(atlas), np.max(atlas)

(0.0, 246.0)

In [10]:
def MNI2XYZ(mni):
    x =  - mni[0] + 90
    y = mni[1] + 126
    z = mni[2] + 72
    return [x,y,z]


In [11]:
# Test
mni = [30,-26,23]
MNI2XYZ(mni)

[60, 100, 95]

In [44]:
def HO_region(out):
    out_new = []
    i = 0
    for entry in out:
        if 'No label found' in entry:
            return ''
        if (i == 0) and ('Cerebral' in entry):
            return ''
        if '%' in entry:
            out_new.append(entry)
            i=i+1
        else:
            out_new[i-1] = out_new[i-1] + '_'+ entry 

    return out_new[0]     


In [16]:
def getHOAtlasRegions(coord):
    
    
    
    import subprocess

    p1 = subprocess.Popen(["atlasquery", "-a", "Harvard-Oxford Cortical Structural Atlas", "-c", coord ], stdout=subprocess.PIPE)
    p2 = subprocess.Popen(["atlasquery", "-a", "Harvard-Oxford Subcortical Structural Atlas", "-c", coord], stdout=subprocess.PIPE)

    out1 = p1.communicate()
    out2 = p2.communicate()

    out = []


    out1 = str(out1[0]).split('<br>')[1].split(',')
    out2 = str(out2[0]).split('<br>')[1].split(',')
    
    if int(coord.split(',')[0]) < 0:
        hemisphere = '_L'
    else:
        hemisphere = '_R'
    return [HO_region(out1) + hemisphere ,  HO_region(out2)+ hemisphere ]

    
# d = {'ROI':[roi], 'Voxels':[row[0]], 'Max_tvalue':[row[1]], 'MNI(X Y Z)':[coord], 'Harvard-Oxford Cortical Structural Atlas':[str(out1[0]).split('>')[3]], 'Harvard-Oxford Subcortical Structural Atlas':str(out2[0]).split('>')[3], 'Juelich Histological Atlas':str(out3[0]).split('>')[3]}

    
    

In [ ]:
def roi(atlas_path, seedMNI):
    
    atlas = nib.load(atlas_path).get_data()
    x,y,z = MNI2XYZ(seedMNI)
    
    roi_number = atlas[x,y,z] + 1 # (1 to 246)
    
    
    return roi_number, roiNameBrainnetome, roiNameHO

In [2]:
# numSeeds = 246
import pandas as pd

class _queryBrainnetomeROI:
    def __init__(self, atlas_path, atlasRegionsDescrpPath):
        self.atlas_path = atlas_path
        self.numSeeds = None
        self.atlasRegionsDescrpPath = atlasRegionsDescrpPath
#         self.ROI_dictionary = None
        
    
    def brainnetomeQuery(self):

        atlas = nib.load(self.atlas_path).get_data()
        df = pd.read_excel(self.atlasRegionsDescrpPath)

        self.numSeeds = int(np.max(atlas))

        df = df.as_matrix(['Label ID.L', 'Label ID.R','Left and Right Hemisphere','Unnamed: 5', 'lh.MNI(X,Y,Z)', 'rh.MNI(X,Y,Z)'])

        seedInfo = np.empty((self.numSeeds + 1, 3),dtype=object)

        for i,j,region,label,lcoord,rcoord in df:
            seedInfo[i,:] = i, region.split('_')[0] + '_' + label, lcoord
            seedInfo[j,:] = j, region.split('_')[0] + '_' + label, rcoord

    #     seedInfo[1:]

        # Create a dictionary of seedInfo so that it can be accessed by roi number

        ROI_dictionary = {}
        for i in seedInfo[1:]:
            ROI_dictionary[i[0]] = i[1:]
            
        return ROI_dictionary


    #     d[1]

    def queryDict(self, lis): # lis: List that contains ROI numbers (1 to 246)
        
        ROI_dictionary = self.brainnetomeQuery()
        return_regions = []
        
        
        if isinstance(lis,list):
            for i in lis:
        #         return_regions.append(str(i)+"_"+d[i][0])
                return_regions.append(ROI_dictionary[i][0])
        else:
            i = lis
            return_regions.append(ROI_dictionary[i][0])
        
        return return_regions

    

In [9]:
# numSeeds = 246
import pandas as pd

class queryBrainnetomeROI:
    def __init__(self, atlas_path, atlasRegionsDescrpPath):
        self.atlas_path = atlas_path
        self.numSeeds = None
        self.atlasRegionsDescrpPath = atlasRegionsDescrpPath
#         self.ROI_dictionary = None
        
    
    def brainnetomeQuery(self):

        atlas = nib.load(self.atlas_path).get_data()
        df = pd.read_excel(self.atlasRegionsDescrpPath)

        self.numSeeds = int(np.max(atlas))

        df = df.as_matrix(['Lobe','Gyrus','Label ID.L', 'Label ID.R','Left and Right Hemisphere','Unnamed: 5', 'lh.MNI(X,Y,Z)', 'rh.MNI(X,Y,Z)'])
        seedInfo = np.empty((self.numSeeds + 1, 5),dtype=object)

        for lobe, gyrus, i,j,region,label,lcoord,rcoord in df:
            seedInfo[i,:] = i,lobe, gyrus, region.split('_')[0] + '_' + label, lcoord
            seedInfo[j,:] = j,lobe, gyrus, region.split('_')[0] + '_' + label, rcoord

    #     seedInfo[1:]

        # Create a dictionary of seedInfo so that it can be accessed by roi number

        ROI_dictionary = {}
        for i in seedInfo[1:]:
            ROI_dictionary[i[0]] = i[1:]
            
        return ROI_dictionary


    #     d[1]

    def queryDict(self, lis): # lis: List that contains ROI numbers (1 to 246)
        
        ROI_dictionary = self.brainnetomeQuery()
        return_regions = []
        
        
        if isinstance(lis,list):
            for i in lis:
        #         return_regions.append(str(i)+"_"+d[i][0])
                return_regions.append(ROI_dictionary[i])
        else:
            i = lis
            return_regions.append(ROI_dictionary[i])
        
        return return_regions

    

In [10]:
atlas_path = 'BNA-maxprob-thr0-1mm.nii.gz'
# atlasRegionsDescrpPath = '/home/varun/Projects/fmri/Autism-Connectome-Analysis-brain_connectivity/atlas/BNA_subregions.xlsx'
atlasRegionsDescrpPath = 'brainnetomeAtlas/BNA_subregions_machineReadable.xlsx'
q = queryBrainnetomeROI(atlas_path, atlasRegionsDescrpPath)

In [11]:
q.queryDict([245,246]) # Returns Lobe, Gyrus, Region and MNI Coordinates of the query ROI Number

[array(['Subcortical Nuclei', 'Tha, Thalamus',
        'Tha_lPFtha, lateral pre-frontal thalamus', '-11, -14, 2 '],
       dtype=object), array(['Subcortical Nuclei', 'Tha, Thalamus',
        'Tha_lPFtha, lateral pre-frontal thalamus', '13, -16, 7 '],
       dtype=object)]

In [39]:
atlasRegionsDescrpPath = 'brainnetomeAtlas/BNA_subregions_machineReadable.xlsx'

pd.read_excel(atlasRegionsDescrpPath)

,Lobe,Gyrus,Left and Right Hemisphere,Label ID.L,Label ID.R,Unnamed: 5,Anatomical and modified Cyto-architectonic descriptions,"lh.MNI(X,Y,Z)","rh.MNI(X,Y,Z)"
0,Frontal Lobe,"SFG, Superior Frontal Gyrus",SFG_L(R)_7_1,1,2,"A8m, medial area 8",NaN,"-5 ,15, 54","7, 16, 54"
1,Frontal Lobe,"SFG, Superior Frontal Gyrus",SFG_L(R)_7_2,3,4,"A8dl, dorsolateral area 8",NaN,"-18, 24, 53","22, 26, 51"
2,Frontal Lobe,"SFG, Superior Frontal Gyrus",SFG_L(R)_7_3,5,6,"A9l, lateral area 9",NaN,"-11, 49, 40","13, 48, 40"
3,Frontal Lobe,"SFG, Superior Frontal Gyrus",SFG_L(R)_7_4,7,8,"A6dl, dorsolateral area 6",NaN,"-18, -1, 65","20, 4, 64"
4,Frontal Lobe,"SFG, Superior Frontal Gyrus",SFG_L(R)_7_5,9,10,"A6m, medial area 6",NaN,"-6, -5, 58","7, -4, 60"
5,Frontal Lobe,"SFG, Superior Frontal Gyrus",SFG_L(R)_7_6,11,12,"A9m,medial area 9",NaN,"-5, 36, 38","6, 38, 35"
6,Frontal Lobe,"SFG, Superior Frontal Gyrus",SFG_L(R)_7_7,13,14,"A10m, medial area 10",NaN,"-8, 56, 15","8, 58, 13"
7,Frontal Lobe,"MFG, Middle Frontal Gyrus",MFG_L(R)_7_1,15,16,"A9/46d, dorsal area 9/46",NaN,"-27, 43, 31","30, 37, 36"
8,Frontal Lobe,"MFG, Middle Frontal Gyrus",MFG_L(R)_7_2,17,18,"IFJ, inferior frontal junction",NaN,"-42, 13, 36","42, 11, 39"
9,Frontal Lobe,"MFG, Middle Frontal Gyrus",MFG_L(R)_7_3,19,20,"A46, area 46",NaN,"-28, 56, 12","28, 55, 17"


In [85]:
def MNI2XYZ2mm(mni):

    x =  math.floor((- mni[0] + 90)/2.0)
    y = math.floor((mni[1] + 126)/2.0)
    z = math.floor((mni[2] + 72)/2.0)
    return [x,y,z]

In [90]:
MNI2XYZ2mm([80,-75,-42])

[5, 25, 15]

In [25]:
def connectivityCheckCSV(csvPath):
        # read CSV file
        df = pd.read_csv(csvPath)
        
        
        # Extract the seedMNI and UnderConnectedMNI column
        # Call self.connectivityCheck to get the code and connectivity string
        # Create another CSV file with rows - 
        # | SeedName, SeedMNI, UnderConnectedName, UnderConnectedMNI, ConsistentCheck,
        #  OverConnectedName, OverConnectedMNI, ConsistentCheck  |
        
        return df  
csvPath = 'Literature_Survey_ResultsSheet3.csv'
df = connectivityCheckCSV(csvPath)

In [26]:
df

,SeedName,SeedMNI,UnderConnectivityName,UnderConnectivityMNI,OverConnectivityName,OverConnectivityMNI
0,PCC,-5 -35 41,Left medial prefrontal,−20 38 18,NaN,NaN
1,PCC,-5 -35 42,Left medial prefrontal,−46 56 8,NaN,NaN
2,PCC,-5 -35 43,Left medial prefrontal,−16 22 40,NaN,NaN
3,PCC,-5 -35 44,Right medial prefrontal,10 34 18,NaN,NaN
4,PCC,-5 -35 45,Left temporal lobe,−50 12 -14,NaN,NaN
5,PCC,-5 -35 46,Right temporal lobe,50 -58 8,NaN,NaN
6,PCC,-5 -35 47,Right temporal lobe,38 16 -22,NaN,NaN
7,PCC,-5 -35 48,Right temporal lobe,62 8 -6,NaN,NaN
8,PCC,-5 -35 49,Right temporal lobe,34 2 -14,NaN,NaN
9,PCC,-5 -35 50,Left superior frontal gyrus,−24 42 18,NaN,NaN


In [33]:
dfMatrix = df.as_matrix(['SeedName', 'SeedMNI', 'UnderConnectivityName', 'UnderConnectivityMNI', 'OverConnectivityName',\
             'OverConnectivityMNI']) 
dfMatrix

array([['PCC', '-5 -35 41', 'Left medial prefrontal', '−20 38 18', nan,
        nan],
       ['PCC', '-5 -35 42', 'Left medial prefrontal', '−46 56 8', nan,
        nan],
       ['PCC', '-5 -35 43', 'Left medial prefrontal', '−16 22 40', nan,
        nan],
       ['PCC', '-5 -35 44', 'Right medial prefrontal', '10 34 18', nan,
        nan],
       ['PCC', '-5 -35 45', 'Left temporal lobe', '−50 12 -14', nan, nan],
       ['PCC', '-5 -35 46', 'Right temporal lobe', '50 -58 8', nan, nan],
       ['PCC', '-5 -35 47', 'Right temporal lobe', '38 16 -22', nan, nan],
       ['PCC', '-5 -35 48', 'Right temporal lobe', '62 8 -6', nan, nan],
       ['PCC', '-5 -35 49', 'Right temporal lobe', '34 2 -14', nan, nan],
       ['PCC', '-5 -35 50', 'Left superior frontal gyrus', '−24 42 18',
        nan, nan],
       ['PCC', '-5 -35 51', 'Left superior frontal gyrus', '−16 18 56',
        nan, nan],
       ['PCC', '-5 -35 52', 'Right superior frontal gyrus', '12 22 50',
        nan, nan],
       ['PCC'

In [36]:
res = '-5 -35 41'.split(' ')
list(map(int, res))

[-5, -35, 41]

In [34]:
df_new = pd.DataFrame(columns=['SeedName', 'SeedMNI', 'UnderConnectivityName', 'UnderConnectivityMNI','ConsistencyUC', 'OverConnectivityName',\
             'OverConnectivityMNI', 'ConsistencyOC'])
for row in dfMatrix:
    seedName, seedMNI, UnderConnectivityName, underConnectedMNI,OverConnectivityName, OverConnectivityMNI = row
    
    seedMNIint = seedMNI.split(' ')
    seedMNIint = list(map(int, seedMNIint))
    
    if not np.isnan(underConnectedMNI):
        underConnectedMNIint = underConnectedMNI.split(' ')
        underConnectedMNIint = list(map(int, underConnectedMNIint))
        ConsistencyUC = 
        
        
    
    df_new = df_new.append({'SeedName':SeedName,
                            'SeedMNI':SeedMNI,
                            'UnderConnectivityName':UnderConnectivityName,
                            'UnderConnectivityMNI':UnderConnectivityMNI,
                            'ConsistencyUC':ConsistencyUC, 
                            'OverConnectivityName':OverConnectivityName,
                            'OverConnectivityMNI':OverConnectivityMNI,
                            'ConsistencyOC':ConsistencyOC
      }, ignore_index=True)
    print(seedMNI)
# Seed MNI:
# seedMNI = dfMatrix[:,1]

-5 -35 41
-5 -35 42
-5 -35 43
-5 -35 44
-5 -35 45
-5 -35 46
-5 -35 47
-5 -35 48
-5 -35 49
-5 -35 50
-5 -35 51
-5 -35 52
-5 -35 53
-5 -35 54
-5 -35 55
51 -52 21
51 -52 22
51 -52 23
51 -52 24
51 -52 25
51 -52 26
 47 −60 4
 48 −60 4
 49 −60 4
 50 −60 4
 51 −60 4
 52 −60 4
 53 −60 4
 54 −60 4
 55 −60 4
 56 −60 4
 57 −60 4
 58 −60 4
 59 −60 4
 60 −60 4
 61 −60 4
 62 −60 4
 63 −60 4
 64 −60 4
 65 −60 4
 66 −60 4
 67 −60 4
 68 −60 4
 69 −60 4
 70 −60 4
47 −60 4
48 −60 4
49 −60 4
50 −60 4
51 −60 4
52 −60 4
54 -55 26
55 -55 26
56 -55 26
9 9 -8

9 9 -8

9 9 -8

-9 9 -8

-9 9 -8

-9 9 -8

10 15 0
11 15 0
-10 15 0
-10 15 1
13 15 9

13 15 9

13 15 9


28 1 3


28 1 3


28 1 3

25 8 6
25 8 7
25 8 8
20 12 -3
-13 15 9

-13 15 9

-13 15 9


-28 1 3


-28 1 3


-28 1 3

-25 8 6
-25 8 7
-25 8 8
-20 12 -3
-6 52 -2
-6 52 -3
-6 52 -4
-8 -56 26
9 9 -8
28 1 3
-28 1 3
25 8 6
-25 8 6 
-25 8 7
-25 8 8
20  12 -3
-20 12 -3
 − 52 14 − 2
 − 52 14 − 3
 − 52 14 − 4
 − 52 14 − 5
 − 52 14 − 6
 − 52 14 − 7
 − 52 14 − 8
 

In [11]:
import subprocess
import math

class queryAtlasROI:
    '''
    Common base class for all the functions to query the atlas
    '''
    def __init__(self, atlas_path):
        self.atlas_path = atlas_path
        
    def MNI2XYZ1mm(self, mni):
        x =  - mni[0] + 90
        y = mni[1] + 126
        z = mni[2] + 72
        return [x,y,z]
    
    def MNI2XYZ2mm(self, mni):
            
        x =  math.floor((- mni[0] + 90)/2.0)
        y = math.floor((mni[1] + 126)/2.0)
        z = math.floor((mni[2] + 72)/2.0)
        return [x,y,z]
    
    def HO_region(self, out):
        out_new = []
        i = 0
        for entry in out:
            if 'No label found' in entry:
                return ''
            if (i == 0) and ('Cerebral' in entry):
                return ''
            if '%' in entry:
                out_new.append(entry)
                i=i+1
            else:
                out_new[i-1] = out_new[i-1] + '_'+ entry 

        return out_new[0]   
    
    def getHOAtlasRegions(self, coord):

        p1 = subprocess.Popen(["atlasquery", "-a", "Harvard-Oxford Cortical Structural Atlas", "-c", coord ], stdout=subprocess.PIPE)
        p2 = subprocess.Popen(["atlasquery", "-a", "Harvard-Oxford Subcortical Structural Atlas", "-c", coord], stdout=subprocess.PIPE)

        out1 = p1.communicate()
        out2 = p2.communicate()

        out = []


        out1 = str(out1[0]).split('<br>')[1].split(',')
        out2 = str(out2[0]).split('<br>')[1].split(',')

        if int(coord.split(',')[0]) < 0:
            hemisphere = '_L'
        else:
            hemisphere = '_R'
        return [self.HO_region(out1) + hemisphere ,  self.HO_region(out2)+ hemisphere ]
    
    def queryDict_HO(self, coord,roi_number=None):
        
#         assert isinstance(lis,list)
        
        
        
        return_cortical_regions = []
        return_subcortical_regions = []
#         for i in lis:
        x,y,z = coord 
        coord = str(x) +','+ str(y)+ ','+ str(z) 
#         print(getHOAtlasRegions(d[i][1]))
        cortical,subcortical = self.getHOAtlasRegions(coord)
    
        if (cortical == '_L') or (cortical == '_R'):
            cortical = ''
        else:
            if roi_number != None:
                cortical = str(roi_number)+ "_" + cortical
                

        if (subcortical == '_L') or (subcortical == '_R'):
            subcortical = ''
        else:
            if roi_number != None:
                subcortical = str(roi_number)+ "_" + subcortical

        return_cortical_regions.append(cortical)
        return_subcortical_regions.append(subcortical)
        return return_cortical_regions+return_subcortical_regions
    
    def getROI(self, seedMNI, mm = 1 ,extraAtlasObject = None, atlas_path = None):

        atlas = nib.load(self.atlas_path).get_data()
        
        if mm == 1:
            x,y,z = self.MNI2XYZ1mm(seedMNI)
        else:
            x,y,z = self.MNI2XYZ2mm(seedMNI)
            
        roi_number = atlas[x,y,z] # (1 to 246)
        
        # Now send this ROI number to queryDict function to query about the Brainnetome region
        
        roiNameHO = self.queryDict_HO(seedMNI,roi_number)
        
        if extraAtlasObject == None:
            return roi_number, roiNameHO
        else:
            roiNameBrainnetome = extraAtlasObject.queryDict(roi_number)
            return roi_number, roiNameBrainnetome, roiNameHO
        

    

# TODO
## class CheckConnectivity
Attributes: 
* logq 2mm file


Methods:
* UnderConnectivityCheck(seedMNI,MNICoordinates)
* OverConnectivityCheck(seedMNI,MNICoordinates)
    * Inputs:
        * seedMNI
        * MNICoordinates
    * Output:
        * 1: UnderConnected 
        * 2: OverConnected
        * 0: Not Significant 
* checkConnectivity
    * Inputs:
        * CSV with | Seed, UnderConnectedMNI, OverConnectedMNI | as columns.
    * Output:
        * input like CSV with multiple columns specifiying 0,1 or 2
        
    


In [12]:
class checkConnectivity:
    '''
    queryAtlasObj gives the method to get the ROI number
    queryExtraAtlasObj gives the method to access Brainnetome atlas related methods 
    '''
    def __init__(self,brainLogqFile, queryAtlasObj, queryExtraAtlasObj):
        self.brainLogqFile = brainLogqFile
        self.ROIbrain = nib.load(self.brainLogqFile).get_data()
        self.queryAtlasObj=queryAtlasObj
        self.queryExtraAtlasObj=queryExtraAtlasObj
        
    def MNI2XYZ2mm(self, mni):
            
        x =  math.floor((- mni[0] + 90)/2.0)
        y = math.floor((mni[1] + 126)/2.0)
        z = math.floor((mni[2] + 72)/2.0)
        return [x,y,z]    
        
        
    def connectivityCheck(self,seedMNI, voxelMNI, alpha):
        x,y,z = self.MNI2XYZ2mm(voxelMNI)
        roi = int(self.queryAtlasObj.getROI(seedMNI, self.queryExtraAtlasObj )[0])
        print('ROI: ',roi)
        
        
        value = self.ROIbrain[x,y,z,roi-1]
        
        print('Value at given MNI is :',value)
        if value < 0:
            if abs(value) >= alpha:
                code,connectivity = 1, 'Underconnected'
            else:
                code,connectivity = 0, 'n.s'
        if value >= 0:
            if abs(value) >= alpha:
                code, connectivity = 2, 'Overconnected'
            else:
                code, connectivity =  0, 'n.s'
        
        return code, connectivity
    
    def connectivityCheckCSV(self,csvPath,alpha):
            # read CSV file
        df = pd.read_csv(csvPath)
        dfMatrix = df.as_matrix(['SeedName', 'SeedMNI', 'UnderConnectivityName', 'UnderConnectivityMNI', 'OverConnectivityName',\
             'OverConnectivityMNI'])

        df_new = pd.DataFrame(columns=['SeedName', 'SeedMNI', 'UnderConnectivityName', 'UnderConnectivityMNI','ConsistencyUC', 'OverConnectivityName',\
             'OverConnectivityMNI', 'ConsistencyOC'])
        for row in tqdm(dfMatrix):
            seedName, seedMNI, UnderConnectivityName, UnderConnectedMNI,OverConnectivityName, OverConnectedMNI = row
            print(seedName, seedMNI, UnderConnectivityName, UnderConnectedMNI,OverConnectivityName, OverConnectedMNI)

            seedMNI = seedMNI.strip('\n')

            
            seedMNIint = seedMNI.split(' ')
            seedMNIint = [x for x in seedMNIint if x != '']
            
            print('Seed: ',seedMNIint)

            seedMNIint = list(map(int, seedMNIint))

            if not pd.isna(UnderConnectedMNI):
                UnderConnectedMNI = UnderConnectedMNI.strip('\n')
                UnderConnectedMNIint = UnderConnectedMNI.split(' ')
                UnderConnectedMNIint = [x for x in UnderConnectedMNIint if x != '']
                print(UnderConnectedMNIint)
                UnderConnectedMNIint = list(map(int, UnderConnectedMNIint))

                ConsistencyUC = q3.connectivityCheck(seedMNIint, UnderConnectedMNIint, alpha)[0]

            else:
                ConsistencyUC = UnderConnectedMNI

            if not pd.isna(OverConnectedMNI):
                OverConnectedMNI = OverConnectedMNI.strip('\n')
                OverConnectedMNIint = OverConnectedMNI.split(' ')
                OverConnectedMNIint = [x for x in OverConnectedMNIint if x != '']
                print(OverConnectedMNIint)
                OverConnectedMNIint = list(map(int, OverConnectedMNIint))
                
                ConsistencyOC = q3.connectivityCheck(seedMNIint, OverConnectedMNIint, alpha)[0]

            else:
                ConsistencyOC = OverConnectedMNI




            df_new = df_new.append({'SeedName':seedName,
                                    'SeedMNI':seedMNI,
                                    'UnderConnectivityName':UnderConnectivityName,
                                    'UnderConnectivityMNI':UnderConnectedMNI,
                                    'ConsistencyUC':ConsistencyUC, 
                                    'OverConnectivityName':OverConnectivityName,
                                    'OverConnectivityMNI':OverConnectedMNI,
                                    'ConsistencyOC':ConsistencyOC
              }, ignore_index=True)

        # Extract the seedMNI and UnderConnectedMNI column
        # Call self.connectivityCheck to get the code and connectivity string
        # Create another CSV file with rows - 
        # | SeedName, SeedMNI, UnderConnectedName, UnderConnectedMNI, ConsistentCheck,
        #  OverConnectedName, OverConnectedMNI, ConsistentCheck  |

        fileName = 'connectivityResults.csv'
        df_new.to_csv(fileName)

        return df_new, fileName  
    
    
    def checkDensity(seedMNI, voxelMNI):
        # Get the Brainnetome atlas ROI number form seedMNI
        
        # Get the HO atlas ROI number from voxelMNI [if a voxel is in both HO cortical and subcortical then give priority 
        # to subcortical]
        # Find the density of hyper / hypo connected voxels in the atlas
        
        
    


In [13]:
atlas_path = 'BNA-maxprob-thr0-1mm.nii.gz'
atlasRegionsDescrpPath = '/home/varun/Projects/fmri/Autism-Connectome-Analysis-brain_connectivity/atlas/BNA_subregions.xlsx'


brainLogqFile = 'map_logq_2mm.nii.gz'
queryAtlasObj = queryAtlasROI(atlas_path)
queryExtraAtlasObj =  queryBrainnetomeROI(atlas_path, atlasRegionsDescrpPath)
q3 = checkConnectivity(brainLogqFile, queryAtlasObj, queryExtraAtlasObj)


In [20]:
seedMNI = [-18, 24, 53 ]#[30, -30, -18 ]#[-28, -32, -18] 
voxelMNI = [-54, -24, -22 ]#[2,56,-22] # [44,-46,50]#[34,-46,70] #  [34, -44 ,72]
alpha = 1.3
q3.connectivityCheck(seedMNI, voxelMNI, alpha)

ROI:  3
Value at given MNI is : -0.016887205


(0, 'n.s')

In [17]:
alpha = 1.3
csvPath = 'Literature_Survey_ResultsSheet3.csv'
q3.connectivityCheckCSV(csvPath,alpha)


  0%|          | 0/111 [00:00<?, ?it/s]

PCC -5 -35 41 Left medial prefrontal -20 38 18 nan nan
Seed:  ['-5', '-35', '41']
['-20', '38', '18']



  1%|          | 1/111 [00:02<05:27,  2.97s/it]

ROI:  175
Value at given MNI is : -0.3994488
PCC -5 -35 41 Left medial prefrontal -46 56 8 nan nan
Seed:  ['-5', '-35', '41']
['-46', '56', '8']



  2%|▏         | 2/111 [00:05<05:21,  2.95s/it]

...

 99%|█████████▉| 110/111 [05:54<00:03,  3.22s/it]

ROI:  39
Value at given MNI is : -0.0044498933
Broca (IFG) left  -52 14 -2 Left cerebellum crus I -34 -62 -32 nan nan
Seed:  ['-52', '14', '-2']
['-34', '-62', '-32']


100%|██████████| 111/111 [05:56<00:00,  3.22s/it]

ROI:  39
Value at given MNI is : 0.002611508


(                              SeedName     SeedMNI  \
 0                                  PCC   -5 -35 41   
 1                                  PCC   -5 -35 41   
 2                                  PCC   -5 -35 41   
 3                                  PCC   -5 -35 41   
 4                                  PCC   -5 -35 41   
 5                                  PCC   -5 -35 41   
 6                                  PCC   -5 -35 41   
 7                                  PCC   -5 -35 41   
 8                                  PCC   -5 -35 41   
 9                                  PCC   -5 -35 41   
 10                                 PCC   -5 -35 41   
 11                                 PCC   -5 -35 41   
 12                                 PCC   -5 -35 41   
 13                                 PCC   -5 -35 41   
 14                                 PCC   -5 -35 41   
 15                                rTPJ   51 -52 21   
 16                                rTPJ   51 -52 21   
 17       

In [ ]:
s = ['', '47', '-60', '4']


In [56]:

def connectivityCheckCSV(csvPath):
    # read CSV file
    df = pd.read_csv(csvPath)
    dfMatrix = df.as_matrix(['SeedName', 'SeedMNI', 'UnderConnectivityName', 'UnderConnectivityMNI', 'OverConnectivityName',\
         'OverConnectivityMNI'])

    df_new = pd.DataFrame(columns=['SeedName', 'SeedMNI', 'UnderConnectivityName', 'UnderConnectivityMNI','ConsistencyUC', 'OverConnectivityName',\
         'OverConnectivityMNI', 'ConsistencyOC'])
    for row in tqdm(dfMatrix):
        seedName, seedMNI, UnderConnectivityName, UnderConnectedMNI,OverConnectivityName, OverConnectedMNI = row
        print(seedName, seedMNI, UnderConnectivityName, UnderConnectedMNI,OverConnectivityName, OverConnectedMNI)

        seedMNIint = seedMNI.split(' ')
        seedMNIint = list(map(int, seedMNIint))

        if not pd.isna(UnderConnectedMNI):
            UnderConnectedMNIint = UnderConnectedMNI.split(' ')
            print(UnderConnectedMNIint)
            UnderConnectedMNIint = list(map(int, UnderConnectedMNIint))

            ConsistencyUC = q3.connectivityCheck(seedMNIint, UnderConnectedMNIint, alpha)[0]

        else:
            ConsistencyUC = UnderConnectedMNI

        if not pd.isna(OverConnectedMNI):
            OverConnectedMNIint = OverConnectedMNI.split(' ')
            OverConnectedMNIint = list(map(int, OverConnectedMNIint))

            ConsistencyOC = q3.connectivityCheck(seedMNIint, OverConnectedMNIint, alpha)[0]

        else:
            ConsistencyOC = OverConnectedMNI




        df_new = df_new.append({'SeedName':seedName,
                                'SeedMNI':seedMNI,
                                'UnderConnectivityName':UnderConnectivityName,
                                'UnderConnectivityMNI':UnderConnectedMNI,
                                'ConsistencyUC':ConsistencyUC, 
                                'OverConnectivityName':OverConnectivityName,
                                'OverConnectivityMNI':OverConnectedMNI,
                                'ConsistencyOC':ConsistencyOC
          }, ignore_index=True)

    # Extract the seedMNI and UnderConnectedMNI column
    # Call self.connectivityCheck to get the code and connectivity string
    # Create another CSV file with rows - 
    # | SeedName, SeedMNI, UnderConnectedName, UnderConnectedMNI, ConsistentCheck,
    #  OverConnectedName, OverConnectedMNI, ConsistentCheck  |
    
    fileName = 'connectivityResults.csv'
    df_new.to_csv(fileName)

    return df_new, fileName  


csvPath = 'Literature_Survey_ResultsSheet3.csv'
df = connectivityCheckCSV(csvPath)

In [57]:
df

,SeedName,SeedMNI,UnderConnectivityName,UnderConnectivityMNI,OverConnectivityName,OverConnectivityMNI
0,PCC,-5 -35 41,Left medial prefrontal,-20 38 18,NaN,NaN
1,PCC,-5 -35 42,Left medial prefrontal,-46 56 8,NaN,NaN
2,PCC,-5 -35 43,Left medial prefrontal,-16 22 40,NaN,NaN
3,PCC,-5 -35 44,Right medial prefrontal,10 34 18,NaN,NaN
4,PCC,-5 -35 45,Left temporal lobe,-50 12 -14,NaN,NaN
5,PCC,-5 -35 46,Right temporal lobe,50 -58 8,NaN,NaN
6,PCC,-5 -35 47,Right temporal lobe,38 16 -22,NaN,NaN
7,PCC,-5 -35 48,Right temporal lobe,62 8 -6,NaN,NaN
8,PCC,-5 -35 49,Right temporal lobe,34 2 -14,NaN,NaN
9,PCC,-5 -35 50,Left superior frontal gyrus,-24 42 18,NaN,NaN


In [58]:
dfMatrix = df.as_matrix(['SeedName', 'SeedMNI', 'UnderConnectivityName', 'UnderConnectivityMNI', 'OverConnectivityName',\
             'OverConnectivityMNI']) 
dfMatrix

array([['PCC', '-5 -35 41', 'Left medial prefrontal', '-20 38 18', nan,
        nan],
       ['PCC', '-5 -35 42', 'Left medial prefrontal', '-46 56 8', nan,
        nan],
       ['PCC', '-5 -35 43', 'Left medial prefrontal', '-16 22 40', nan,
        nan],
       ['PCC', '-5 -35 44', 'Right medial prefrontal', '10 34 18', nan,
        nan],
       ['PCC', '-5 -35 45', 'Left temporal lobe', '-50 12 -14', nan, nan],
       ['PCC', '-5 -35 46', 'Right temporal lobe', '50 -58 8', nan, nan],
       ['PCC', '-5 -35 47', 'Right temporal lobe', '38 16 -22', nan, nan],
       ['PCC', '-5 -35 48', 'Right temporal lobe', '62 8 -6', nan, nan],
       ['PCC', '-5 -35 49', 'Right temporal lobe', '34 2 -14', nan, nan],
       ['PCC', '-5 -35 50', 'Left superior frontal gyrus', '-24 42 18',
        nan, nan],
       ['PCC', '-5 -35 51', 'Left superior frontal gyrus', '-16 18 56',
        nan, nan],
       ['PCC', '-5 -35 52', 'Right superior frontal gyrus', '12 22 50',
        nan, nan],
       ['PCC'

In [59]:
res = '-5 -35 41'.split(' ')
list(map(int, res))

[-5, -35, 41]

In [63]:
df_new = pd.DataFrame(columns=['SeedName', 'SeedMNI', 'UnderConnectivityName', 'UnderConnectivityMNI','ConsistencyUC', 'OverConnectivityName',\
             'OverConnectivityMNI', 'ConsistencyOC'])
for row in dfMatrix:
    seedName, seedMNI, UnderConnectivityName, UnderConnectedMNI,OverConnectivityName, OverConnectedMNI = row
    print(seedName, seedMNI, UnderConnectivityName, UnderConnectedMNI,OverConnectivityName, OverConnectedMNI)
    
    seedMNIint = seedMNI.split(' ')
    seedMNIint = list(map(int, seedMNIint))
    
    if not pd.isna(UnderConnectedMNI):
        UnderConnectedMNIint = UnderConnectedMNI.split(' ')
        print(UnderConnectedMNIint)
        UnderConnectedMNIint = list(map(int, UnderConnectedMNIint))
        
        ConsistencyUC = q3.connectivityCheck(seedMNIint, UnderConnectedMNIint, alpha)[0]
        
    else:
        ConsistencyUC = UnderConnectedMNI
        
    if not pd.isna(OverConnectedMNI):
        OverConnectedMNIint = OverConnectedMNI.split(' ')
        OverConnectedMNIint = list(map(int, OverConnectedMNIint))
        
        ConsistencyOC = q3.connectivityCheck(seedMNIint, OverConnectedMNIint, alpha)[0]
    
    else:
        ConsistencyOC = OverConnectedMNI
        
        
        
    
    df_new = df_new.append({'SeedName':seedName,
                            'SeedMNI':seedMNI,
                            'UnderConnectivityName':UnderConnectivityName,
                            'UnderConnectivityMNI':UnderConnectedMNI,
                            'ConsistencyUC':ConsistencyUC, 
                            'OverConnectivityName':OverConnectivityName,
                            'OverConnectivityMNI':OverConnectedMNI,
                            'ConsistencyOC':ConsistencyOC
      }, ignore_index=True)
    print(seedMNI)
# Seed MNI:
# seedMNI = dfMatrix[:,1]

PCC -5 -35 41 Left medial prefrontal -20 38 18 nan nan
['-20', '38', '18']
ROI:  175
Value at given MNI is : -0.3994488
-5 -35 41
PCC -5 -35 42 Left medial prefrontal -46 56 8 nan nan
['-46', '56', '8']
ROI:  185
Value at given MNI is : 0.068246305
-5 -35 42
PCC -5 -35 43 Left medial prefrontal -16 22 40 nan nan
['-16', '22', '40']
ROI:  185
Value at given MNI is : 0.12570661
-5 -35 43
PCC -5 -35 44 Right medial prefrontal 10 34 18 nan nan
['10', '34', '18']
ROI:  185
Value at given MNI is : -0.9247803
-5 -35 44
PCC -5 -35 45 Left temporal lobe -50 12 -14 nan nan
['-50', '12', '-14']
ROI:  185
Value at given MNI is : -0.0974411
-5 -35 45
PCC -5 -35 46 Right temporal lobe 50 -58 8 nan nan
['50', '-58', '8']


KeyboardInterrupt: 

In [55]:
UnderConnectedMNIint = ['20', '38', '18']
list(map(int, UnderConnectedMNIint))

[-20, 38, 18]

In [93]:
abs(-6.4)

6.4

In [81]:
q2 = queryAtlasROI(atlas_path)

In [83]:
seedMNI = [30, -30, -18 ]#[-28, -32, -18]
voxelMNI = [34, -44 ,72]
q2.getROI(seedMNI,q)

(114.0,
 ['PhG_TL, area TL (lateral PPHC, posterior parahippocampal gyrus)'],
 ['114.0_57% Parahippocampal Gyrus_ posterior division_R', ''])

['PhG_TL, area TL (lateral PPHC, posterior parahippocampal gyrus)']

In [59]:
x,y,z = -28, -32, -18 
coord = str(x) +','+ str(y)+ ','+ str(z) 
q2.getHOAtlasRegions(coord)

['61% Parahippocampal Gyrus_ posterior division_L', '_L']

In [79]:
coord
q2.queryDict_HO([-28, -32, -18],5)

['5_61% Parahippocampal Gyrus_ posterior division_L', '']